In [1]:
# Libraries
import os
import sqlite3
import re
import time
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('..\data\Bleifrei_TS_raw_excel.csv',
                 sep=';',
                 encoding='latin-1')

In [3]:
# Extract values from 'price_raw' strings
brand = []
addresse = []
for i in df['Adresse']:
    d1 = i.split("\r\n\r\n")
    brand.append(d1[0])
    try:
        addresse.append(d1[1])
    except:
        addresse.append("keine")

# Save as new variable in the pandas data frame
df['brand'] = brand
df['addresse'] = addresse

df

,web-scraper-order,web-scraper-start-url,Adresse,Art,Datum,Preis,Preis2,brand,addresse
0,1671561789-2,https://www.benzin-preis.ch/suche?next=2640,"Alpina Tankstelle\r\n\r\nDorfstrasse 42, 7563 ...",Bleifrei,21.09.2020,"1,140 sFr","1,640 sFr",Alpina Tankstelle,"Dorfstrasse 42, 7563 Samnaun Dorf"
1,1671561791-3,https://www.benzin-preis.ch/suche?next=2639,"Tamoil\r\n\r\nGsteigstr. 174, 3785 Gsteig b. G...",Bleifrei,04.07.2022,"2,230 sFr","1,870 sFr",Tamoil,"Gsteigstr. 174, 3785 Gsteig b. Gstaad"
2,1671561794-4,https://www.benzin-preis.ch/suche?next=2638,"Agrola\r\n\r\nKantonsstr. 50, 3930 Eyholz",Bleifrei,04.07.2022,"2,230 sFr","1,870 sFr",Agrola,"Kantonsstr. 50, 3930 Eyholz"
3,1671561796-5,https://www.benzin-preis.ch/suche?next=2637,"Garage Blatter AG\r\n\r\nEnnet Brücke 7, 3930 ...",Bleifrei,04.07.2022,"2,230 sFr","1,870 sFr",Garage Blatter AG,"Ennet Brücke 7, 3930 Visp"
4,1671561798-6,https://www.benzin-preis.ch/suche?next=2636,"Garage Gasparini\r\n\r\nVeia Granda, 7440 Andeer",Bleifrei,11.07.2022,"2,230 sFr","1,930 sFr",Garage Gasparini,"Veia Granda, 7440 Andeer"
...,...,...,...,...,...,...,...,...,...
2615,1671567999-2637,https://www.benzin-preis.ch/suche?next=5,"Ruedi Rüssel\r\n\r\nLuzernstr. 60, 4553 Subingen",Bleifrei,22.12.2020,"1,260 sFr","1,710 sFr",Ruedi Rüssel,"Luzernstr. 60, 4553 Subingen"
2616,1671568001-2638,https://www.benzin-preis.ch/suche?next=4,"Dorfgarage Willi AG\r\n\r\nRheintalstrasse 59,...",Bleifrei,22.12.2020,"1,250 sFr","1,700 sFr",Dorfgarage Willi AG,"Rheintalstrasse 59, 5325 Leibstadt"
2617,1671568004-2639,https://www.benzin-preis.ch/suche?next=3,"Garage Jaques\r\n\r\nRoute de Lausanne, 1260 Nyon",Bleifrei,24.07.2016,"1,250 sFr",NaN,Garage Jaques,"Route de Lausanne, 1260 Nyon"
2618,1671568006-2640,https://www.benzin-preis.ch/suche?next=2,Hansruedi Julmi Garage & Tankstelle\r\n\r\nBie...,Bleifrei,27.11.2020,"1,250 sFr","1,730 sFr",Hansruedi Julmi Garage & Tankstelle,"Bielstrasse 85, 2542 Pieterlen"


In [4]:
counter=0;
count=0;

for i in df['addresse']:
    if i == "keine":
        df = df.drop(labels=counter, axis=0)
        count = count + 1
    counter = counter + 1
    
print(count)

16


In [5]:
# Extract values from 'price_raw' strings
price = []
for i in df['Preis']:
    d1 = "".join(c for c in i if  c.isdecimal())
    price.append((int(d1) / 1000))

# Save as new variable in the pandas data frame
df['bleifrei'] = price

In [6]:
df.drop(columns=['Preis','Art','Adresse','Preis2','web-scraper-order','web-scraper-start-url'], inplace=True, axis=1)

In [7]:
df

,Datum,brand,addresse,bleifrei
0,21.09.2020,Alpina Tankstelle,"Dorfstrasse 42, 7563 Samnaun Dorf",1.14
1,04.07.2022,Tamoil,"Gsteigstr. 174, 3785 Gsteig b. Gstaad",2.23
2,04.07.2022,Agrola,"Kantonsstr. 50, 3930 Eyholz",2.23
3,04.07.2022,Garage Blatter AG,"Ennet Brücke 7, 3930 Visp",2.23
4,11.07.2022,Garage Gasparini,"Veia Granda, 7440 Andeer",2.23
...,...,...,...,...
2615,22.12.2020,Ruedi Rüssel,"Luzernstr. 60, 4553 Subingen",1.26
2616,22.12.2020,Dorfgarage Willi AG,"Rheintalstrasse 59, 5325 Leibstadt",1.25
2617,24.07.2016,Garage Jaques,"Route de Lausanne, 1260 Nyon",1.25
2618,27.11.2020,Hansruedi Julmi Garage & Tankstelle,"Bielstrasse 85, 2542 Pieterlen",1.25


In [8]:
df.to_csv('..\data\bleifrei.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)